## Asmuth-Bloom Secret Sharing

In [1]:
from math import sqrt
from functools import reduce
import random

In [2]:
def generate_primes(secret,n):
    '''
    params- start:The first prime in the generated mignotte sequence, n:The number of primes in the mignotte sequence
    returns- A list of primes (potentially a mignotte sequence)
    '''
    asmuth_bloom = []
    primes=[2]
    p=3
    while(len(asmuth_bloom)<n):
        stop_iter=sqrt(p)
        i=0
        flag=True
        while(primes[i]<=stop_iter):
            if(p%primes[i]==0):
                flag=False
                break
            elif(i>=len(primes)):
                break
            i+=1
            
        if(flag):
            primes.append(p)
            if(p>secret):
                asmuth_bloom.append(p)
        p+=2
    return(asmuth_bloom)

def check_asmuth_bloom_seq(seq,n,k):
    '''
    params- seq:The potential Asmuth-Bloom sequence, n:number of participants, k:number of shares available
    returns- True if the sequence is an Asmuth-Bloom Sequence and the product of the first k primes in the sequence
    '''
    p1=1
    for i in range(1,k+1):
        p1*=seq[i]
    p2=seq[0]
    for i in range(n-k+2,n+1):
        p2*=seq[i]
    
    return((p1>p2,p1))

def generate_shares(secret,n,k):
    '''
    params- secret: the key to be split, n: the number of partipants, 
            k: the threshold of shares required to reconstruct the secret
    returns- The Asmuth-Bloom Sequence generated, and n shares of the secret
    '''
    M=generate_primes(secret,1)
    for i in generate_primes(M[0]*3,n):
        M.append(i)
    is_asmuth_bloom_seq=check_asmuth_bloom_seq(M,n,k)
    
    print("Asmuth Bloom Sequence :")
    print(M)
    
    test,p1=is_asmuth_bloom_seq
    
    if(test):
        print("Asmuth-Bloom Condition Satisfied")
    else:
        print("Not a valid Asmuth-Bloom Sequence")
        
    product_seq=reduce(lambda x,y:x*y,M[1::])
    
    y=secret
    while(True):
            a=random.randint(1,p1//2-1)
            y=secret+a*M[0]
            if(0<=y<=p1):
                break
    shares=[y%M[i] for i in range(1,n+1)]
    return(M,shares)

def extended_euclid(a,b):
    '''
    Find the gcd of params a,b and computes integers x,y such that ax+by=gcd(a,b)
    '''
    if(b==0):
        return(a,1,0)
    d,x,y=extended_euclid(b,a%b)
    return((d,y,x-(a//b)*y))


def chinese_remainder_theorem(M,shares,k):
    '''
    params- M: the primes corresponding to the shares available, shares: the shares available,
            k: Number of shares available
    returns- The unique solution modulo prod(M) to the system of equations described by y=shares_i(mod M_i)
    '''
    if(len(shares)<k):
        return(-1)
    prod=reduce(lambda x,y: x*y, M)
    m=[prod//M[i] for i in range(0,k)]
    c=[extended_euclid(m[i],M[i])[1]%M[i] for i in range(0,k)]
    A=[c[i]*shares[i]*m[i] for i in range(k)]
    return(sum(A)%prod)



### A (10,3) Asmuth-Bloom Example <br>
(n,k)=(10,3) <br>
Secret=123456

In [3]:
n,k=5,3
S=123456
print("(n,k) :",(n,k))
print("Secret :",S)

share_no=list(range(n))
available_shares=random.sample(share_no,k)

M,shares=generate_shares(S,n,k)
M_fil=M[1::]

M_s=[M_fil[i] for i in available_shares]
s=[shares[i] for i in available_shares]
print("Shares: ")
print(shares)

y1=chinese_remainder_theorem(M_s,s,k)
if y1==-1:
    print("Insufficient Shares Available")
else:
    print("Recovered Secret",y1%M[0])

(n,k) : (5, 3)
Secret : 123456
Asmuth Bloom Sequence :
[123457, 370373, 370387, 370399, 370411, 370421]
Asmuth-Bloom Condition Satisfied
Shares: 
[335650, 164479, 208553, 340878, 254545]
Recovered Secret 123456
